In [23]:
import PyPDF2
import os
import re
import pandas as pd

In [24]:
def extract_info(pdf_file_path):
    data={
        "Document title": "nan",
        "Version number": "nan",
        "Reference/Register number": "nan",
        "Document Type": "nan",
        "Issue Date": "nan",
        "Review Date": "nan",
        "Author": "nan"
    }
    with open(pdf_file_path,"rb") as f:
        file_reader=PyPDF2.PdfFileReader(f)
        text=""
        for x in range(file_reader.numPages):
            text+=file_reader.getPage(x).extract_text()
        data["Document title"] = file_reader.getDocumentInfo().title
        data["Author"] = file_reader.getDocumentInfo().author
        #data["Issue Date"] = file_reader.getDocumentInfo().creation_date
        data["Issue Date"]=re.search(r"Issue Date:\s*(.*)",text,re.IGNORECASE)
        data["Reference/Register number"]=re.search(r"Reference/Register No:\s*(.*)",text,re.IGNORECASE)
        data["Version number"]=re.search(r"Version  No:\s*(.*)",text,re.IGNORECASE)
        data["Document Type"]=re.search(r"Document Type:\s*(.*)",text,re.IGNORECASE)
        data["Review Date"]=re.search(r"Review Date:\s*(.*)",text,re.IGNORECASE)                      
    for y in data:
        if data[y] is not None and isinstance(data[y], re.Match):
            data[y] = data[y].group(1).strip()
        else:
            data[y] = "nan"

    return data      
def data_processing(file_path):    
    pdf_files = [file for file in os.listdir(file_path) if file.endswith(".pdf")]
    all_data = []
    for file in pdf_files:
        files_path = os.path.join(file_path, file)
        data = extract_info(files_path)  
        all_data.append(data)
    return all_data

def csv_file(data, outputfile):
    df=pd.DataFrame(data)
    df.to_csv(outputfile,index=False)
file_directory = r"C:\Users\Dell\Desktop\Panda"
outputfile = "new_data_file.csv"
all_data = data_processing(file_directory)
csv_file(all_data, outputfile)
